# 1. 데이터 준비 및 확인

# 2. 데이터 정규화 (Normalization)

# 3. 가중치 부여 및 종합 점수 계산

# 4. 점수 분포 분석 및 기준 설정
	•	계산된 종합 점수를 기반으로 지역별 점수 분포 파악 (예: 히스토그램, 박스플롯)
	•	상위 몇 %를 고독사 위험 지역으로 선정할지 기준 설정 (예: 상위 20% 지역 선정)


# 5. 결과 해석 및 시각화
	•	위험도가 높은 지역 리스트 추출
	•	지도 시각화(예: Folium, geopandas)로 위험지역 시각적으로 표현
	•	보고서용 표나 그래프 작성

# 6. 정책 제안 및 추가 분석 (옵션)
	•	위험지역에 맞춤형 복지 대책 제안
	•	시간별, 연도별 변화 분석(추세 분석)
	•	추가 변수(예: 경제수준, 의료기관 수 등) 포함하여 모델 고도화


# 1. 데이터 준비 및 확인

In [10]:
import pandas as pd

one_family_df = pd.read_csv('data/독거노인가구비율_시도_시_군_구__20250615135552.csv')
public_health_center = pd.read_csv('data/보건복지부현황.csv', encoding='cp949')
elderly_welfare = pd.read_csv('data/보건복지부_노인복지 생활시설 수 및 생활자 현황_시설 종류별_시도별_20231231.csv', encoding='cp949')
mental_health = pd.read_excel('data/기초정신건강복지센터+현황.xlsx')

In [3]:
!pip install openpyxl


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [5]:
region_counts = public_health_center.groupby('시도').size().reset_index(name='지역별_개수')
print(region_counts)

         시도  지역별_개수
0   강원특별자치도     256
1       경기도     346
2      경상남도     421
3      경상북도     557
4     광주광역시      24
5     대구광역시      27
6     대전광역시      20
7     부산광역시      44
8     서울특별시      59
9   세종특별자치시      19
10    울산광역시      25
11    인천광역시      69
12     전라남도     573
13  전북특별자치도     409
14  제주특별자치도      67
15     충청남도     410
16     충청북도     272


# 2. 데이터 정규화 (Normalization)


	•	X : 원래 값
	•	X_min : 해당 변수의 최소값
	•	X_max : 해당 변수의 최대값
	•	X_norm : 0과 1 사이로 변환된 값


In [76]:
# 독거 노인 비율 계산
one_family = {}

regions = one_family_df['행정구역별'].unique()

for region in regions:
    value = one_family_df[one_family_df['행정구역별'] == region]['65세이상 1인가구<br>(A) (가구)'].values[0]
    one_family[region] = int(value)  # ← 여기서 np.int64 → int 로 변환
del one_family['행정구역별']
del one_family['전국']
print(one_family)

min_val = min(one_family.values())
max_val = max(one_family.values())

family_normalized = {region: (val - min_val) / (max_val - min_val) for region, val in one_family.items()}
print(family_normalized)


{'서울특별시': 334348, '부산광역시': 174540, '대구광역시': 109018, '인천광역시': 102510, '광주광역시': 56049, '대전광역시': 54110, '울산광역시': 36987, '세종특별자치시': 7691, '경기도': 406282, '강원특별자치도': 91601, '충청북도': 79135, '충청남도': 105574, '전북특별자치도': 106075, '전라남도': 123303, '경상북도': 158914, '경상남도': 168365, '제주특별자치도': 23605}
{'서울특별시': 0.8195292919308264, '부산광역시': 0.418597007960541, '대구광역시': 0.2542129651698107, '인천광역시': 0.23788545150291904, '광주광역시': 0.12132235800607641, '대전광역시': 0.1164577223268965, '울산광역시': 0.07349889987480901, '세종특별자치시': 0.0, '경기도': 1.0, '강원특별자치도': 0.21051654452810023, '충청북도': 0.17924137775313542, '충청남도': 0.24557252923422757, '전북특별자치도': 0.2468294567614422, '전라남도': 0.290051707138395, '경상북도': 0.3793939150658194, '경상남도': 0.4031049371410795, '제주특별자치도': 0.03992563806006658}


In [77]:
# 보거소 비율 계산

health_count = {}

regions = public_health_center['시도'].unique()

for region in regions :
    value = public_health_center[public_health_center['시도']==region].count()
    health_count[region] = int(value.iloc[0])

min_val = min(health_count.values())
max_val = max(health_count.values())

health_normalized = {region: (val - min_val) / (max_val - min_val) for region, val in health_count.items()}
print(health_normalized)

{'서울특별시': 0.07220216606498195, '충청북도': 0.4566787003610108, '충청남도': 0.7057761732851986, '전북특별자치도': 0.703971119133574, '전라남도': 1.0, '경상북도': 0.9711191335740073, '경상남도': 0.7256317689530686, '제주특별자치도': 0.08664259927797834, '세종특별자치시': 0.0, '부산광역시': 0.04512635379061372, '대구광역시': 0.01444043321299639, '인천광역시': 0.09025270758122744, '광주광역시': 0.009025270758122744, '대전광역시': 0.0018050541516245488, '울산광역시': 0.010830324909747292, '경기도': 0.5902527075812274, '강원특별자치도': 0.427797833935018}


In [92]:
# 노인복지 생활 시설 수

elderly_welfare = elderly_welfare[elderly_welfare['연도'] == 2023]
elderly_welfare.head()

welfare = {}

regions = elderly_welfare['시도'].unique()

# 양로시설
# 노인공동생활가정
# 노인요양시설
# 노인요양공동생활가정
weights = [1, 1.5, 3, 2]  # 양로, 공동생활, 요양, 요양공동생활

for region in regions:
    value = []
    value.append(elderly_welfare[elderly_welfare['시도'] == region]['노인주거복지시설_양로시설_시설수'].values[0])
    value.append(elderly_welfare[elderly_welfare['시도'] == region]['노인주거복지시설_노인공동생활가정_시설수'].values[0])
    value.append(elderly_welfare[elderly_welfare['시도'] == region]['노인의료복지시설_노인요양시설_시설수'].values[0])
    value.append(elderly_welfare[elderly_welfare['시도'] == region]['노인의료복지시설_노인요양공동생활가정_시설수'].values[0])

    weighted_sum = sum([v * w for v, w in zip(value, weights)])
    welfare[region] = int(weighted_sum)


min_val = min(welfare.values())
max_val = max(welfare.values())

welfare_normalized = {region: (val - min_val) / (max_val - min_val) for region, val in welfare.items()}
print(welfare_normalized)
rename_map = {
    '서울Seoul': '서울특별시',
    '부산Busan': '부산광역시',
    '대구Daegu': '대구광역시',
    '인천Incheon': '인천광역시',
    '광주Gwangju': '광주광역시',
    '대전Daejeon': '대전광역시',
    '울산Ulsan': '울산광역시',
    '세종Sejong': '세종특별자치시',
    '경기Gyeonggi': '경기도',
    '강원Gangwon': '강원특별자치도',
    '충북Chungbuk': '충청북도',
    '충남Chungnam': '충청남도',
    '전북Jeonbuk': '전라북도',
    '전남Jeonnam': '전라남도',
    '경북Gyeongbuk': '경상북도',
    '경남Gyeongnam': '경상남도',
    '제주Jeju': '제주특별자치도'
}

welfare_normalized = {rename_map[k]: v for k, v in welfare_normalized.items()}

print(welfare_normalized)

{'서울Seoul': 0.18899882609424787, '부산Busan': 0.043266812007378835, '대구Daegu': 0.10363910783162837, '인천Incheon': 0.22807311755827603, '광주Gwangju': 0.03940969310749623, '대전Daejeon': 0.06070769746771759, '울산Ulsan': 0.01911789367767902, '세종Sejong': 0.0, '경기Gyeonggi': 1.0, '강원Gangwon': 0.1435519034043267, '충북Chungbuk': 0.13382525574375315, '충남Chungnam': 0.15210464531276202, '전북Jeonbuk': 0.10649002180110682, '전남Jeonnam': 0.14137179272178432, '경북Gyeongbuk': 0.19134663759852424, '경남Gyeongnam': 0.11839678014422271, '제주Jeju': 0.023142713399295658}
{'서울특별시': 0.18899882609424787, '부산광역시': 0.043266812007378835, '대구광역시': 0.10363910783162837, '인천광역시': 0.22807311755827603, '광주광역시': 0.03940969310749623, '대전광역시': 0.06070769746771759, '울산광역시': 0.01911789367767902, '세종특별자치시': 0.0, '경기도': 1.0, '강원특별자치도': 0.1435519034043267, '충청북도': 0.13382525574375315, '충청남도': 0.15210464531276202, '전라북도': 0.10649002180110682, '전라남도': 0.14137179272178432, '경상북도': 0.19134663759852424, '경상남도': 0.11839678014422271, '제주특별자치도': 0

In [94]:
# 기초 정신건강 센터 현황

mental_health.head()

mental = {}

regions = mental_health['시·도'].unique()

for region in regions :
    value = mental_health[mental_health['시·도']==region].count()
    mental[region] = int(value.iloc[0])

min_val = min(mental.values())
max_val = max(mental.values())

mental_normalized = {region: (val - min_val) / (max_val - min_val) for region, val in mental.items()}
rename_map = {
    '서울': '서울특별시',
    '부산': '부산광역시',
    '대구': '대구광역시',
    '인천': '인천광역시',
    '광주': '광주광역시',
    '대전': '대전광역시',
    '울산': '울산광역시',
    '세종': '세종특별자치시',
    '경기': '경기도',
    '강원': '강원특별자치도',
    '충북': '충청북도',
    '충남': '충청남도',
    '전북': '전라북도',
    '전남': '전라남도',
    '경북': '경상북도',
    '경남': '경상남도',
    '제주': '제주특별자치도'
}

mental_normalized = {rename_map.get(k, k): v for k, v in mental_normalized.items()}

print(mental_normalized)


{'서울특별시': 0.5609756097560976, '부산광역시': 0.34146341463414637, '대구광역시': 0.14634146341463414, '인천광역시': 0.21951219512195122, '광주광역시': 0.0975609756097561, '대전광역시': 0.07317073170731707, '울산광역시': 0.07317073170731707, '세종특별자치시': 0.0, '경기도': 1.0, '강원특별자치도': 0.3902439024390244, '충청북도': 0.2926829268292683, '충청남도': 0.34146341463414637, '전라북도': 0.2926829268292683, '전라남도': 0.4878048780487805, '경상북도': 0.5609756097560976, '경상남도': 0.43902439024390244, '제주특별자치도': 0.0}


In [75]:
# 고독사 발생현황
# pdf 자료
lonely_death = {
   "서울특별시": 559,
    "부산광역시": 287,
    "대구광역시": 183,
    "인천광역시": 208,
    "광주광역시": 94,
    "대전광역시": 104,
    "울산광역시": 72,
    "세종특별자치시": 8,
    "경기도": 922,
    "강원특별자치도": 167,
    "충청북도": 167,
    "충청남도": 183,
    "전라북도": 126,
    "전라남도": 120,
    "경상북도": 186,
    "경상남도": 235,
    "제주특별자치도": 51
}

min_val = min(lonely_death.values())
max_val = max(lonely_death.values())

death_normalized = {region: (val - min_val) / (max_val - min_val) for region, val in lonely_death.items()}
print(death_normalized)

{'서울특별시': 0.6028446389496718, '부산광역시': 0.3052516411378556, '대구광역시': 0.19146608315098468, '인천광역시': 0.2188183807439825, '광주광역시': 0.09409190371991247, '대전광역시': 0.1050328227571116, '울산광역시': 0.0700218818380744, '세종특별자치시': 0.0, '경기도': 1.0, '강원특별자치도': 0.1739606126914661, '충청북도': 0.1739606126914661, '충청남도': 0.19146608315098468, '전라북도': 0.12910284463894967, '전라남도': 0.12253829321663019, '경상북도': 0.19474835886214442, '경상남도': 0.24835886214442013, '제주특별자치도': 0.047045951859956234}


# 3. 가중치 부여 및 종합 점수 계산


In [98]:
# family_normalized
# health_normalized
# welfare_normalized
# mental_normalized
# death_normalized

health_sorted = dict(sorted(health_normalized.items(), key=lambda item: item[1], reverse=True))
welfare_sorted = dict(sorted(welfare_normalized.items(), key=lambda item: item[1], reverse=True))
mental_sorted = dict(sorted(mental_normalized.items(), key=lambda item: item[1], reverse=True))
print(health_sorted)
print(welfare_sorted)
print(mental_sorted)

# 1. DataFrame 생성
df = pd.DataFrame({
    'family': family_normalized,
    'health': health_normalized,
    'welfare': welfare_normalized,
    'mental': mental_normalized,
    'death': death_normalized
})

# 2. 인프라 항목 반전 (많을수록 좋은 항목 → 적을수록 위험)
reverse_cols = ['health', 'welfare', 'mental']
for col in reverse_cols:
    df[col] = 1 - df[col]

# 3. 가중치 조정
weights = {
    'family': 4,    # 독거노인 수 (많으면 위험)
    'health': 6,    # 보건소 (많으면 안전 → 1-값, 가중치 높게)
    'welfare': 6,   # 복지시설 (많으면 안전 → 1-값, 가중치 높게)
    'mental': 6,    # 정신건강센터 (많으면 안전 → 1-값, 가중치 높게)
    'death': 4      # 고독사 수 (많으면 위험)
}

# 4. 가중치 적용
for col in df.columns:
    df[col] *= weights[col]

# 5. 종합 점수 계산
df['total_score'] = df.sum(axis=1)

# 6. 정렬
df = df.sort_values(by='total_score', ascending=False)

# 결과 확인
print(df[['total_score']])

{'전라남도': 1.0, '경상북도': 0.9711191335740073, '경상남도': 0.7256317689530686, '충청남도': 0.7057761732851986, '전북특별자치도': 0.703971119133574, '경기도': 0.5902527075812274, '충청북도': 0.4566787003610108, '강원특별자치도': 0.427797833935018, '인천광역시': 0.09025270758122744, '제주특별자치도': 0.08664259927797834, '서울특별시': 0.07220216606498195, '부산광역시': 0.04512635379061372, '대구광역시': 0.01444043321299639, '울산광역시': 0.010830324909747292, '광주광역시': 0.009025270758122744, '대전광역시': 0.0018050541516245488, '세종특별자치시': 0.0}
{'경기도': 1.0, '인천광역시': 0.22807311755827603, '경상북도': 0.19134663759852424, '서울특별시': 0.18899882609424787, '충청남도': 0.15210464531276202, '강원특별자치도': 0.1435519034043267, '전라남도': 0.14137179272178432, '충청북도': 0.13382525574375315, '경상남도': 0.11839678014422271, '전라북도': 0.10649002180110682, '대구광역시': 0.10363910783162837, '대전광역시': 0.06070769746771759, '부산광역시': 0.043266812007378835, '광주광역시': 0.03940969310749623, '제주특별자치도': 0.023142713399295658, '울산광역시': 0.01911789367767902, '세종특별자치시': 0.0}
{'경기도': 1.0, '서울특별시': 0.5609756097560976, '경상북도

# 4. 점수 분포 분석 및 기준 설정
상위 20%,
15이상 매우 위험 : 진한 빨강 #b10026
12~15 위험 : 빨강 #e31a1c
9~12 보통 : 주황 #fd8d3c
6~9 다소 낮음 : 연주황 #fecc5c
3~6 낮음 : 연노랑 #ffffb2
0~3 매우 낮음 : 회색 #0f0f0f0
